We import needed libraries

In [1]:
import numpy as np
from sklearn import datasets
from typing import Union

We define necessary functions

In [2]:
def unique_with_same_order(a: np.ndarray, axis=None):
    return a[np.sort(np.unique(a, return_index=True, axis=axis)[1])]

def convert_to_one_hot(a: Union[np.ndarray, list]):
    if isinstance(a, list):
        a = np.array(a)
    return np.vstack([a == v for v in unique_with_same_order(a, axis=0)]).T.astype(np.float64)

def standard_scaling(X):
    return (X - X.mean(axis=0)) / X.std(axis=0)

def train_test_split(X = None, population_size: int = None, split: float = 0.2):
    """
    params:
    X: np.ndarray or pd.DataFrame passed to infer population_size
    population_size: can be passed instead of X
    split: fraction of dataset that goes into test set
    """
    if X is not None:
        population_size = X.shape[0]
        
    shuffled_index = np.random.permutation(np.arange(population_size))
    test_size = int(population_size * split)
    test_index = shuffled_index[:test_size]
    train_index = shuffled_index[test_size:]
    return train_index, test_index

def softmax(a, rowwise=True):
    if rowwise:
        return (np.exp(a).T / np.exp(a).T.sum(0)).T
    else:
        return np.exp(a) / np.exp(a).sum()
    
def predict_proba(params, X):
    return softmax(params.dot(X.T).T)
    
def predict(params, X):
    return np.argmax(
        predict_proba(params, X), 
        axis=1)

def cross_entropy(y, y_hat, X = None, gradient: bool = False):
    """
    params
    y: ground_truth labels encoded as 1-hot
    y_hat: same shape as y, but with probabilities of classes instead of ground_truth
    X: required only if gradient set to True 
    gradient: True if function should return a gradient of cross_entropy
    """
    if gradient:
        if X is None:
            raise ValueError("X parameter cannot be None when calculating gradient")
        return ((y_hat - y).T @ X) / y_hat.shape[0]
    else:
        return -(1/y.shape[0]) * np.multiply(y, np.log(y_hat)).sum()
    
def accuracy(output, target):
    return np.sum(output == target) / target.shape[0]

We define model implementation

In [3]:
class SoftmaxRegressionClassifier:
    def __init__(self):
        self.params: np.ndarray = None
        self.classes: np.ndarray = None
        
    def fit(self, X, y, X_val, y_val, learning_rate: float, n_epochs: int = 10):
        n_params = X.shape[1]
        self.classes = unique_with_same_order(y, axis=0)
        self._init_params(self.classes.shape[0], n_params)
        y_oh = convert_to_one_hot(y)
        
        for epoch_i in range(n_epochs):
            y_hat = self.predict_proba(X)
            loss = cross_entropy(y_oh, y_hat)
            gradient = cross_entropy(y_oh, y_hat, X, gradient=True)
            self.params -= learning_rate * gradient
            
    def _init_params(self, n_classes, n_params):
        self.params = np.random.normal(size=(n_classes, n_params))
    
    def predict_proba(self, X):
        if self.params is None:
            raise AssertionError("You cannot predict before you fit the model")
        return softmax(self.params.dot(X.T).T)
    
    def predict(self, X):
        return self.classes[np.argmax(
            self.predict_proba(X), 
            axis=1)]

We download the dataset

In [4]:
iris = datasets.load_iris()

We preprocess the data

In [5]:
X = iris.data
y = iris.target_names[iris.target]

population_size = X.shape[0]
X_scaled = standard_scaling(X)

We split the data

In [6]:
val_split = 0.2 
train_index, val_index = train_test_split(X_scaled, split=0.2)
X_train, y_train = X_scaled[train_index], y[train_index]
X_val, y_val = X_scaled[val_index], y[val_index]

In [7]:
sr_cls = SoftmaxRegressionClassifier()
sr_cls.fit(X_train, y_train, X_val, y_val, learning_rate=0.05, n_epochs=200)

In [8]:
val_accuracy = accuracy(sr_cls.predict(X_val), y_val)
train_accuracy = accuracy(sr_cls.predict(X_train), y_train)
print('val_accuracy:', val_accuracy.round(3))
print('train_accuracy:', train_accuracy.round(3))
bias = 1 - train_accuracy
variance = 1 - val_accuracy - bias
print('bias error:', bias.round(3))
print('variance error:', variance.round(3))

val_accuracy: 0.8
train_accuracy: 0.833
bias error: 0.167
variance error: 0.033
